# **Включите тёмную тему для корректного отображения рисунков: Settings -> Theme -> Jupyter Dark**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pyspark.sql.functions as F
from pyspark import StorageLevel

# Подготовка

## Spark Контекст

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("execution-plan")
        .config("spark.scheduler.mode", "FIFO")
        .master("local[4]")
        .getOrCreate()
)
sc = spark.sparkContext

После старта сессии `Spark UI` становится доступным на порту [`4040`](http://localhost:4040)

## Данные

In [ ]:
!mkdir -p /tmp/taxi
!unzip -o -d /tmp/taxi ./data/taxi.zip

Файл `taxi.parquet` необходимо разбить на несколько частей, чтобы можно было обрабатывать каждую одновременно:

In [ ]:
spark.read.parquet("/tmp/taxi/taxi.parquet").repartition(4).write.mode("overwrite").parquet("/tmp/taxi_many")

## Запросы

In [ ]:
hello_world_df = spark.sql("select 'Hello, World!' as message")
hello_world_df.show()

In [ ]:
taxi_single_partition_df = spark.read.parquet("/tmp/taxi/taxi.parquet")
taxi_single_partition_df.groupBy("passenger_count").count().show()

In [ ]:
taxi_many_partitions_df = spark.read.parquet("/tmp/taxi_many")
taxi_many_partitions_df.groupBy("passenger_count").count().show()

# Spark UI

## Заголовок

![main menu](../imgs/spark-ui-header.drawio.PNG)

Заголовок разделен на 2 части:

1. имя приложения: отображает значение, переданное в [`Builder#appName`](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.SparkSession.builder.appName.html?highlight=appname#pyspark.sql.SparkSession.builder.appName);
1. главное меню: позволяет переключаться между экранами:
    - Jobs - список задач,
    - Stages - список стадий, на которые разбиваются задачи,
    - Storage - информация о кэше,
    - Environment - итоговая конфигурация приложения,
    - Executors - список воркеров в кластере Apache Spark,
    - (Дополнительно) SQL/DataFrame - планы запущенных запросов,
    - (Дополнительно) Structured Streaming - потоковая обработка Apache Kafka при помощи Apache Spark.

Дополнительные пункты главного меню не видны сразу, они появляются, когда приложение обращается к DataFrame API или Structured Streaming API.

## Задачи - Spark Jobs

Первая страница Spark UI - это Spark Jobs. Все операции в Spark делятся на 2 типа:

1. Трансформации (transformations) - операции, которые формируют план запроса (`select`, `where`, `groupBy`, и т.д.). Никакой реальной работы с данными не выполняется;
1. Действия (actions) - операции, которые запускают исполнение сформированного плана (`show`, `count`, и т.д.).

Экран Spark Jobs показывает запущенные действия (actions).

Экран Spark Jobs логически разделен на 3 секции:

1. Общая конфигурация,
1. Event Timeline,
1. Список задач (jobs).

### Общая конфигурация

![jobs](../imgs/spark-ui-jobs-stat.jpg)

Общая конфигурация показывает 3 настройки:

1. Пользователь, от имени которого будет выполняться доступ к ресурсам,
1. Общее время работы приложение: сколько времени прошло с момента старта приложения,
1. Тип планировщика. Отображаемое значение соответствует значению конфига `spark.scheduler.mode`, которое было указано при старте приложения.

#### Тип планировщика

Планировщик является потокобезопасным, а значит в рамках одного Spark приложения/SparkContext могут быть запущены несколько задач (jobs) одновременно из разных потоков. Эта возможность позволяет обрабатывать параллельно обрабатывать запросы от разных клиентов.

По умолчанию, Spark запускает задачи по очереди (`FIFO`), и задача в голове очереди получает приоритетный доступ ко всем ресурсам. Так одна большая задача может сильно снизить пропускную способность (throughput) и повысить среднюю задержку (latency).

При помощи настройки `spark.scheduler.mode` можно организовать более справедливый доступ к ресурсам, если передать значение `FAIR`. Так Spark будет использовать алгоритм Round Robin при выделении ресурсов задачам, а значит все задачи будут получать примерно равное время.

[Documentation](https://spark.apache.org/docs/latest/job-scheduling.html#scheduling-within-an-application)

### Event Timeline

Event Timeline показывает:

1. задачи (job): когда задача была создана, и сколько времени заняла обработка этой задачи,
1. воркеры (executors): появление новых и удаление имеющихся воркеров.

![event timeline](../imgs/spark-ui-event-timeline.jpg)

### Задачи (Jobs)

План запроса представляется в виде графа, который Spark делит на задачи во время обработки. Задача может быть активной или завершенной. Завершенные задачи отображаются в виде таблицы на странице Spark Jobs. Таблица имеет следующие колонки:

1. **Job Id** - номер задачи,
2. **Description** - имя функции, которая запустила задачу,
3. **Submitted** - время создания задачи,
4. **Duration** - общее время работы задачи,
5. **Stages: Succeeded/Total** - количество стадий задачи (граница стадии - широкая операция),
6. **Tasks: Succeeded/Total** - количество заданий (tasks).

![Spark Jobs](../imgs/spark-ui-jobs.jpg)

### Детали задачи

Если нажать на задачу, откроется страница с деталями задачи. На странице можно увидеть следующую информацию:

1. Статистика:
    - **Status** - статус выполнения задачи,
    - **Submitted** - время запуска,
    - **Duration** - время работы как сумма времени работы каждой задачи,
    - **Associated SQL Query** - ссылка на план запроса,
    - **Completed Stages** - количество стадий в текущей задаче.
1. Event Timeline - связанные с задачей события,
2. DAG Visualization - граф вычислений текущей задачи,
3. Информация по стадиям - детальная статистика по каждой стадии.

![Job Details](../imgs/spark-ui-job-details.jpg)

## Стадии (Stages)

Задачи (job) делятся на стадии (stage). **Стадия** - это множество узких операций (можно выполнить по очереди на одном воркере). Границей стадии является широкая операция (`join`, `sort`, `sum`, `avg`, и т.д.). Обычно на границе стадии находится операция `Exchange`, которая запускает перемешивание (shuffle) данных между воркерами.

Каждая стадия имеет уникальный порядковый номер, который является сквозным среди всех задач. Перейти к списку стадий можно двумя способами:

- Выбрать **Stages** в пункте главного меню,
- Нажать на стадию на странице с деталями задачи.

![Spark Stages](../imgs/spark-ui-stages-all.jpg)

На странице со стадиями можно заметить разделение всех стадий на две части:

- **Completed Stages** - стадии, в которых выполнялись вычисления,
- **Skipped Stages** - стадии, которые не были запущены. Одной из причин является активное кэширование результатов, и если Spark видит, что результаты для стадии уже есть, то стадия помечается **skipped**, и вычисления не запускаются.

### Завершенные стадии - Completed Stages

Таблица с завершенными стадиями отображает статистику по запуску стадий. Особый интерес представляют колонки:

- **Input** - объем данных, которые пришли на вход для исполнения стадии. Если стадия читает данные из файла, то его размер будет указан в этой колонке;
- **Output** - объем данных, которые стадия записала куда-либо: файл, таблица, jdbc и т.д.;
- **Shuffle Write** - объем данных, которые стадия отправила другой стадии в результате перемешивания;
- **Shuffle Read** - объем данных, полученных стадией от другой стадии в результате перемешивания (shuffle/exchange).

Анализируя таблицу, можно заметить, как данные перемещались между стадиями в рамках одной задачи:

![Stage Data Flow](../imgs/spark-ui-stages-data-flow.jpg)

### Детали стадии

На странице стадии есть много полезной информации.

В заголовке приводится краткая статистика:

- Время работы стадии (Total Time Across All Tasks): суммарное время работы всех заданий в стадии. Задания могут выполняться параллельно, не смотря на это данное поле будет показывать общую сумму работы всех стадий;
- Локальность данных (Locality Level): данные могут быть находиться в адресном пространстве процесса, на той же машине или на соседней машине. Spark проанализирует, с каким уровнем локальности данных работала каждая задача и покажет эту статистику в этом поле;
- Входящий объем данных (Input Size/Records): сколько всего данных поступило на вход стадии в байтах и строках;
- Исходящий объем данных (Shuffle Write Size / Records): сколько все данных в байтах и строках текущая стадия отправила (exchange/shuffle) другой стадии;
- Идентификатор задачи (Associated Job Ids): идентификатор задачи, к котором относится текущая стадия.

Следующий раздел на странице - это метрики в виде гистограммы. Она реализована в виде таблицы, в которой колонки являются значениями персентиля. Среди метрик можно найти следующие:

- Длительность (Duration) показывает время работы заданий (tasks) по персентилям, а также минимальное и максимальное значение;
- Время сборщика мусора (GC Time) показывает время работы сборщика мусора по персентилям, а также минимальное и максимальное значение;
- Входящий объем (Input Size) показывает сколько данных обрабатывалось каждым заданием по персентилям, а также минимальное и максимальное значение;
- Исходящий объем (Shuffle Write Size) показывает сколько данных каждое задание сформировало в результате с разбиением по персентилям, а также минимальное и максимальное значение;

В самом низу перечислены все задания, которые выполнялись в рамках текущей стадии. Тут можно найти следующую информацию по каждому конкретному заданию:

- уровень локальности данных,
- время работы,
- входящий объем данных,
- исходящий объем данных,
- и т.д.

На картике ниже можно проанализировать одну стадию. Какой вывод можно сделать о производительности? Эффективно ли загружен кластер?

![Stage Details](../imgs/spark-ui-stage-details.jpg)

#### Event Timeline

Event Timeline показывает время запуска заданий, а также дополнительные метрики:

- ожидание запуска (Scheduler Delay) - синий: сколько времени прошло между созданием задания и постановкой его на обработку;
- десериализация (Task Deserializaion Time) - красный: сколько времени заняла десериализация данных для;
- время работы задания (Executor Computing Time) - зеленый: сколько процессорного времени воркера (executor) было затрачено на обработку задания. Нужно стремиться, чтобы это значение было как можно больше;
- время на отправку результатов работы (Shuffle Write Time) - коричневый: сколько времени потребовалось на отправку результатов другим стадиям.

На картинке виден явный перекос (skew) времени работы. Нужно стремиться к тому, чтобы все задачи были примерно одинакового размера. Так кластер будет загружен равномерно.

![Event Timeline](../imgs/spark-ui-stage-event-timeline.jpg)

### Задание

Проанализировать стадию 10. Вопросы:

1. Сколько байт и строк пришло на вход стадии 10?
1. Сколько заданий (task) получили самое большое количество строк на обработку?
1. Какой объем данных (байты и строки) записало задание (task), на вход которому пришло меньше всего строк?
1. Сколько времени заняла обработка стадии 10?
1. Эффективно ли стадия 10 обработала данные?

## Кэширование (Storage)

Пункт "Storage" в главном меню покажет закешированные данные.

### Подготовка данных

In [ ]:
hello_world_df = hello_world_df.cache()
hello_world_df.collect()

In [ ]:
taxi_single_partition_df = taxi_single_partition_df.persist(StorageLevel.DISK_ONLY)
taxi_single_partition_df.count()

### Описание

Список закешированных объектов предстает в виде таблицы с колонками:

- ID - идентификатор объекта в кэше;
- RDD Name - имя закешированного объекта. DataFrame - это просто план запроса, а реальные объекты в Spark представляются в виде RDD, поэтому все закешированные объекты называются RDD;
- Storage Level - где данные сохранены: диск или память;
- Cached Partitons - сколько партиций RDD удалось закешировать;
- Fraction Cached - какой процент RDD удалось закешировать;
- Size in Memory - сколько закешированных данных находится в памяти;
- Size on Disk - сколько закешированных данных находится на диске.

![Storage](../imgs/spark-ui-storage-cache.jpg)

### Детали кэша

Если перейти на страницу одного из закешированных объектов, то можно получить более детальную информацию:

- место хранения (Storage Level) - где размещаются закешированные данные,
- закешированные партиции (Cached Partitions) - сколько партиций удалось закешировать,
- общее число партиций (Total Partitions) - сколько партиций в RDD всего,
- размер объекта в памяти (Memory Size) - сколько данных было закешировано в памяти,
- размер объекта на диске (Disk Size) - сколько данных было закешировано на диске.

![Cache Details](../imgs/spark-ui-storage-cache-details.jpg)

После работы с кэшами обязательно убрать свои объекты из кеша:

In [ ]:
hello_world_df.unpersist()
taxi_single_partition_df.unpersist()

### Задание

Закешировать `taxi_many_partitions_df` с уровнем `StorageLevel.MEMORY_ONLY_2` и проанализировать закешированный объект:

1. Какой процент датафрейма был закеширован?
1. Сколько данных располагается на диске?
1. Сколько данных располагается в памяти?
1. Сколько места в памяти занимает одна партиция датафрейма `taxi_many_partitions_df`?
1. Уберите объект из кэша и убедитесь, что он действительно пропал.

## Environment

Вкладка Environment позволяет посмотреть весь конфиг запущенного приложения, включая те опции, которые явно были указаны при старте:

![Spark Environment](../imgs/spark-ui-env.jpg)

## Воркеры (Executors)

Вкладка Executors показывает статистику по воркерам:

- занятый объем памяти (Storage Memory) включает информацию по занятой памяти как для кэшей, так и для служебных целей;
- занятый объем диска (Disk Used) показывает сколько данных записано на диск. Это может быть либо кэш, либо спилы (spills) необходимые для выполнения сортировки, агреации и др.;
- ядра (Cores) сколько ядер доступно для работы. Одно ядро может обрабатывать одно задание (task);
- задания (tasks) показывает сколько заданий с каким статусом было завершено;
- время сборщика мусора в заданиях (Task Time (GC Time)) показывает сколько времени потребовалось на сборку мусора;
- данные из внешних источников (Input) показывает сколько данных пришло из файлов, jdbc и т.д.;
- входные данные задания (Shuffle Read) показывает объем данных полученных заданиями на входе;
- выходные данные (Shuffle Write) показывает объем данных, которые были отправлены заданиями в результате.

Статистика приводится как общая, так и с разбиением по воркерам.

![Spark Executors](../imgs/spark-ui-executors.jpg)

## SQL / DataFrame

Вкладка SQL / DataFrame показывает планы запросов и краткую статистику по ним.

![imgs](../imgs/spark-ui-sql-dataframe.JPG)

Каждый план запроса имеет порядковый номер, а также ссылку на задачи (job), которые были запущены для исполнения плана. Более детальную информацию о плане можно найти на странице плана, если нажать на поле "Description".

![Plan Details](../imgs/spark-ui-plan.jpg)

## Заключение

Apache Spark предлагает очень мощный интерфейс для анализа запущенных задач и поиска узких мест. Для эффективного использования Spark UI необходимо хорошо понимать основные концепции Apache Spark: запросы (query), задачи (job), стадии (stage), задания (task) и т.д., а так же понимать как они формируются.

## Задание

1. Найти идентификатор самого длинного запроса;
2. Найти объем данных, который первая стадия передала следующей стадии, в самом длинном запросе;
3. Найти самый долгий шаг самого длинного плана запроса. Почему этот шаг такой длинный? Подсказка: нужно посмотреть на стадии и задания.